# 의문
feature는 경기 결과의 일부인데 이걸로 점수를 예측하는 것이 경기 결과 예측이라고 할 수 있는지?

## Main idea
1. Extract winning team's features
2. Learn features with LinearRegression 
3. Test each teams' result and find which team is more similar to learned features
4. predict which team will win

## Import library & Dataset

In [ ]:
pip install numpy pandas seaborn scikit-learn seaborn

In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler

In [27]:
TRAIN_DATA_SRC='./Data/Trainset/Trainset.csv'
TEST_DATA_SRC='./Data/Testset/Testset.csv'
train_df=pd.read_csv(TRAIN_DATA_SRC)
test_df=pd.read_csv(TEST_DATA_SRC)

features=['Possesion','Shots_on_target','Shots','Touches','Passes','Tackles','Clearance','Corners','Offsides','Yellow_cards','Fouls_conceded','Red_cards']

## Preprocess trainset & testset

In [28]:
home_features=[features for features in list(train_df) if 'home' in features and 'team' not in features and 'Score' not in features]
away_features=[features for features in list(train_df) if 'away' in features and 'team' not in features and 'Score' not in features]

winning_train_df=pd.DataFrame(columns=features)
losing_train_df=pd.DataFrame(columns=features)
winning_test_df=pd.DataFrame(columns=features)
losing_test_df=pd.DataFrame(columns=features)

for i,row in train_df.iterrows():
    if row[3]>row[4]:
        winning_train_df.loc[i]=list(row[home_features])
        losing_train_df.loc[i]=list(row[away_features])
    elif row[4]>row[3]:
        winning_train_df.loc[i]=list(row[away_features])
        losing_train_df.loc[i]=list(row[home_features])

for i,row in test_df.iterrows():
    if row[3]>row[4]:
        winning_test_df.loc[i]=list(row[home_features])
        losing_test_df.loc[i]=list(row[away_features])
    elif row[4]>row[3]:
        winning_test_df.loc[i]=list(row[away_features])
        losing_test_df.loc[i]=list(row[home_features])


## PCA

In [48]:
pca=PCA(n_components='mle')
scaler=StandardScaler()
standardized_winning_train_df=pd.DataFrame(scaler.fit_transform(winning_train_df),columns=features)
pca_winning_train_df=pca.fit_transform(standardized_winning_train_df)
pov=pca.explained_variance_
print(pov)

[4.05861052 1.35979163 1.19311343 1.07502493 0.97270886 0.9155342
 0.74296156 0.68236642 0.60193765 0.26418056 0.11750118]


In [55]:
print(sum(pov[:7])/sum(pov))

0.8609793710708921


n_components가 앞 6개의 feature까지는 설명력이 .9 이상이지만 7번째부터 확연히 줄어들기 때문에 PoV가 .86이더라도 feature를 줄이는 것이 나을 것이라고 판단. 
n_components는 6으로 진행.

In [63]:
pca=PCA(n_components=6)
pca_winning_train_df=pd.DataFrame(pca.fit_transform(standardized_winning_train_df))

근데 어떤 feature가 설명력이 높은지 알아야 앞으로 test에서 사용할 수 있음.
아니면 testset을 2007-2022 data를 전부 합쳐서 pca를 진행하고 split_train_test 함수로 쪼개야함.

## Linear Regression

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

To be continued...